# Clearing address from dataframe origin - Tratando base de dados dos endereços do dataframe original. 

In [ ]:
!pip install geopandas  

In [2]:
!pip install shapely

In [3]:
import matplotlib 
import pandas as pd
import geopandas as gpd

In [4]:
from shapely.geometry import Point, Polygon  

In [5]:
from geopandas import geoseries 

- csv file generated from building.ipynb notebook. -- Arquivo csv gerado a partir do notebook building.ipynb. 

In [6]:
ppi_address_ajust = '/content/address_ppi.csv'
ppi_address_ajust = pd.read_csv(ppi_address_ajust, encoding='ISO-8859-1')
ppi_address_ajust.head()

,Unnamed: 0,Empreendimento,UF
0,0,Aeroporto de Salvador/ BA,BA
1,1,Aeroporto de Florianópolis/ SC,SC
2,2,Aeroporto de Fortaleza/ CE,CE
3,3,Aeroporto de Porto Alegre/ RS,RS
4,4,Aeroporto de Maceió/ AL,AL


In [7]:
ppi_address_ajust.drop('Unnamed: 0', axis = 1)

,Empreendimento,UF
0,Aeroporto de Salvador/ BA,BA
1,Aeroporto de Florianópolis/ SC,SC
2,Aeroporto de Fortaleza/ CE,CE
3,Aeroporto de Porto Alegre/ RS,RS
4,Aeroporto de Maceió/ AL,AL
...,...,...
186,"LOTE 9, Atendimento elétrico à região sudeste ...",AM/PA
187,"LOTE 10, Expansão do sistema de transmissão pa...",RJ/ES
188,Rodovia BR-153/282/470/SC e SC-412 \n,SC
189,EF-118 Ferrovia Rio-Vitória (Trecho de Nova Ig...,RJ/ES


In [8]:
ppi_address_ajust.dtypes

Unnamed: 0         int64
Empreendimento    object
UF                object
dtype: object

In [ ]:
ppi_address_ajust.values

In [10]:
ppi_address_ajust['ppi_ajust_cols'] = ppi_address_ajust['Empreendimento'] + '/' + ppi_address_ajust['UF']

In [11]:
ppi_address_ajust

,Unnamed: 0,Empreendimento,UF,ppi_ajust_cols
0,0,Aeroporto de Salvador/ BA,BA,Aeroporto de Salvador/ BA/BA
1,1,Aeroporto de Florianópolis/ SC,SC,Aeroporto de Florianópolis/ SC/SC
2,2,Aeroporto de Fortaleza/ CE,CE,Aeroporto de Fortaleza/ CE/CE
3,3,Aeroporto de Porto Alegre/ RS,RS,Aeroporto de Porto Alegre/ RS/RS
4,4,Aeroporto de Maceió/ AL,AL,Aeroporto de Maceió/ AL/AL
...,...,...,...,...
186,186,"LOTE 9, Atendimento elétrico à região sudeste ...",AM/PA,"LOTE 9, Atendimento elétrico à região sudeste ..."
187,187,"LOTE 10, Expansão do sistema de transmissão pa...",RJ/ES,"LOTE 10, Expansão do sistema de transmissão pa..."
188,188,Rodovia BR-153/282/470/SC e SC-412 \n,SC,Rodovia BR-153/282/470/SC e SC-412 \n/SC
189,189,EF-118 Ferrovia Rio-Vitória (Trecho de Nova Ig...,RJ/ES,EF-118 Ferrovia Rio-Vitória (Trecho de Nova Ig...


In [12]:
pd.set_option('max_rows', 200)
pd.set_option('max_columns', 100)

- 'brazil_states' 

# Tools Geocode, for point location 
 - return GeometryCollection empty

In [13]:
# gpd.tools.geocode(ppi_address_ajust['Empreendimento'], provider = 'nominatim', user_agent='UF Builds', country_bias = 'Brazil')

In [14]:
# ppi_address_ajust['Geom'] = gpd.tools.geocode(ppi_address_ajust['Empreendimento'], provider = 'nominatim', user_agent='UF Builds', country_bias = 'brazil_states')['geometry']

In [15]:
# ppi_address_ajust

In [16]:
# ppi_address_ajust.loc[ppi_address_ajust['Geom'] == 'GEOMETRYCOLLECTION EMPTY', 'Geom'].head()

In [17]:
# ppi_address_ajust.drop_duplicates() # if have, < inplace = True > 

# Geocoders implementation for locations acess - implementação de acesso das localizações usando o Geocoders 
- return None

In [18]:
import geopy.geocoders

In [19]:
from geopy.geocoders import Nominatim

In [20]:
geopy.geocoders.options.default_user_agent = 'my_app/1'

In [21]:
geopy.geocoders.options.default_timeout = 7

In [22]:
geolocator = Nominatim()

In [23]:
print(geolocator.headers)

{'User-Agent': 'my_app/1'}


In [24]:
print(geolocator.timeout)

7


- Test aplication 

In [25]:
# locator = Nominatim(user_agent='myGeocoder')
location = geolocator.geocode('Aeroporto de Salvador/ BA')

In [26]:
print(location)

Aeroporto, Travessa Santos Dumont, Lauro de Freitas, Região Geográfica Imediata de Salvador, Região Metropolitana de Salvador, Região Geográfica Intermediária de Salvador, Bahia, Região Nordeste, 42700000, Brasil


In [27]:
print(location.point)

12 54m 15.1844s S, 38 20m 27.9298s W


In [28]:
location.point

Point(-12.904217899999999, -38.341091608139536, 0.0)

In [29]:
from geopy.extra.rate_limiter import RateLimiter

- Search all address from dataframe

In [30]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [31]:
ppi_address_ajust['ppi_geocode'] = ppi_address_ajust['Empreendimento'].apply(geocode)

In [ ]:
ppi_address_ajust

- plot points of address 

In [33]:
ppi_address_ajust['point'] = ppi_address_ajust['ppi_geocode'].apply(lambda loc: tuple(loc.point) if loc else None)

- ajust None's

In [34]:
pd.set_option('max_rows', 200)
pd.set_option('max_columns', 100)

In [ ]:
ppi_address_ajust

In [ ]:
ppi_address_ajust.isna()

- New position from point values not null. 

In [ ]:
ppi_address_ajust.sort_values(by='point', ignore_index=True)

In [38]:
ppi_address_ajust.index

RangeIndex(start=0, stop=191, step=1)

# Trait DF 

In [39]:
ppi_point = ppi_address_ajust

In [ ]:
ppi_point

In [ ]:
ppi_point.sort_values(by='point', ignore_index=True)

In [42]:
ppi_point = ppi_point.drop(['Unnamed: 0'], axis=1)

In [43]:
ppi_point_true = ppi_point.iloc[:30, :3]

In [ ]:
ppi_point_true

In [45]:
ppi_point_false = ppi_point.iloc[30:191, :3]

In [ ]:
ppi_point_false

In [ ]:
ppi_point_false.reset_index()

# individual search by address 
- Observar o conjunto de caracteres e a forma que se adeque a busa pelo geocode 

  

In [48]:
location_00 = geolocator.geocode('BR-116 - Presidente Dutra	')

In [49]:
print(location_00)

Rodovia Presidente Dutra, Cumbica, Jardim Presidente Dutra, Guarulhos, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 07180041, Brasil


In [50]:
# ppi_point = ppi_address_ajust.drop(['Unnamed: 0',	'Empreendimento',	'UF',	'ppi_ajust_cols'], axis=1)

# Automate address lookup 
- Definir e selecionar uma forma de sanar os problemas que impedem a busca pela geoposição pelo Geocode por célula do dataframe para sanar as deficiências de preenchimento.

 - Softskill - string handlings

In [56]:
import re 

In [65]:
ppi_address_search_00 = ppi_point_false.iloc[0:160, :1]

- For find  condition
>>> 
- equivalent to applying < re.findall() > 
 - Series.str.findall(pat. flags=0) 



In [ ]:
ppi_address_search_00

In [87]:
len(ppi_address_search_00)

160

In [79]:
ppi_terminal = ppi_address_search_00[ppi_address_search_00['Empreendimento'].str.contains('Terminal')]

In [ ]:
ppi_terminal

In [86]:
len(ppi_terminal)

29

In [83]:
ppi_LT = ppi_address_search_00[ppi_address_search_00['Empreendimento'].str.contains('LT')]

In [ ]:
ppi_LT

In [85]:
len(ppi_LT)

54

# To remember codes: 

In [ ]:
#show
# ppi_address_ajust.index.values
# ppi_address_ajust.Geom.values
# type(ppi_address_ajust.Geom.values)

# rename 
# ppi_address_ajust.rename(columns= {'Unnamed: 0': 'Rows_place'}, inplace=True )